# <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:200%; text-align:center;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0">Santa 2023 - Schreier-Sims Algorithm</div>
#### <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:150%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >TABLE OF CONTENTS<br><div>
* [IMPORTS](#1)
* [LOAD DATA](#2)
* [FUNCTIONS](#3)
* [SOLVE](#4)

Attempt at implementing Schreier-Sims Algorithm on the cube puzzles

Using GitHub code from https://github.com/jix/permutation_group_experiments


<a id="1"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" > IMPORTS<br><div> 

In [1]:
import numpy as np
import os
import pandas as pd
from sympy.combinatorics import Permutation, PermutationGroup
import zipfile

from permutation_group_experiments.schreiersims import *

os.chdir('rubiks-cube-NxNxN-solver')
# Print the current working directory
# print("Current Working Directory: ", os.getcwd())

database_file = '../../solutions.db'
solution_method = "dwalton76 NxNxN w/ Schreier-Sim"

<a id="2"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >LOAD DATA<br><div> 

In [2]:
with zipfile.ZipFile('../../../../res/data/santa-2023.zip', 'r') as z:
    
    with z.open('puzzle_info.csv') as f:
        puzzle_info = pd.read_csv(f, index_col = 'puzzle_type')        
                
    with z.open('puzzles.csv') as f:
        puzzles = pd.read_csv(f, index_col='id')
    
    with z.open('sample_submission.csv') as f:
        submission = pd.read_csv(f)

<a id="3"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >FUNCTIONS<br><div>
 

In [3]:
def fmt_large_num(x):
    return re.subn(r'(?<=\d)(?=(\d{3})+$)', ',', str(x))[0]


def add_rubiks_gens(grp, generators):
    for gen in generators:
        grp.add_gen(gen)


def print_all_stats(grp):
    print_sgs_stats(grp)
    print_performance_stats(grp)


def print_sgs_stats(grp):
    print(f"  group order = {fmt_large_num(grp.order())}")
    print(f"  strong generating set base = {grp.base()}")
    print(f"  strong generating set size = {len(grp.generators())}")


def print_performance_stats(grp):
    print(f"  took {fmt_large_num(grp.cfg.stats.products)} group products")
    if grp.cfg.stats.rounds:
        print(f"  took {grp.cfg.stats.rounds} sifting rounds")


In [4]:
def state2ubl(state):
    U_dict = {
        'A': 'U',
        'B': 'F',
        'C': 'R',
        'D': 'B',
        'E': 'L',
        'F': 'D',
    }
    state_split = state.split(';')
    dim = int(np.sqrt(len(state_split) // 6))
    dim_2 = dim**2
    
    s = ''.join([U_dict[f] for f in state_split])
    
    return s[:dim_2] + s[2*dim_2:3*dim_2] + s[dim_2:2*dim_2] + s[5*dim_2:] + s[4*dim_2:5*dim_2] + s[3*dim_2:4*dim_2]


def relabel_NxNxN_index(state):
    # Split the string into individual elements.
    if type(state) != list:
        state_list = state.split(';')
    else:
        state_list = state.copy()
    
    dim = int(np.sqrt(len(state_list) // 6))
    dim_2 = dim**2
    
    for i, num in enumerate(state_list):
        # Replace based on the given criteria.
        if 0 <= num < dim_2:
            state_list[i] = 'A'
        elif dim_2 <= num < 2*dim_2:
            state_list[i] = 'B'
        elif 2*dim_2 <= num < 3*dim_2:
            state_list[i] = 'C'
        elif 3*dim_2 <= num < 4*dim_2:
            state_list[i] = 'D'
        elif 4*dim_2 <= num < 5*dim_2:
            state_list[i] = 'E'
        elif 5*dim_2 <= num < 6*dim_2:
            state_list[i] = 'F'

    return ';'.join(state_list)  

In [5]:
def get_moves(allowed_moves):
    if type(allowed_moves) == str:
        moves = eval(allowed_moves)
    else:
        moves = allowed_moves
        
    for move in list(moves):
        moves['-'+move] = np.argsort(moves[move]).tolist()
    return moves

def apply_moves(initial_state, moves, mmoves):
    state = initial_state.split(";")
    for move_name in mmoves.split('.'):
        # print(move_name)
        state = [state[i] for i in moves[move_name]]
    
    return ';'.join(state)

In [6]:
def sym_rotations(dim, mmoves, new_state, sol_state, num_wildcards, wildcard=False):
    I = ['.'.join([f'{j}{i}' for i in range(dim)]) for j in ['r', 'd', 'f']]
    manipulations = (
            [''] + 
            I + 
            [i1 + '.' + i2 for i1 in I for i2 in I] +
            [i1 + '.' + i2+ '.' + i3 for i1 in I for i2 in I for i3 in I] +
            [i1 + '.' + i2+ '.' + i3 + '.' + i4 for i1 in I for i2 in I for i3 in I for i4 in I]
    )
    
    for rotation in manipulations:
        temp_state = new_state
        if len(rotation) > 0:
            for move in rotation.split('.'):
                temp_state = ';'.join(list(np.asarray(temp_state.split(';'))[np.array(moves[move])]))
        
        if wildcard: 
            total_correct = np.sum([sol_state[i] == ns for i, ns in enumerate(temp_state.split(';'))])
            if total_correct >= 6*(dim**2) - num_wildcards:
                # print(f'solved id: {id}')
                if len(rotation) > 0:
                    mmoves += '.' + rotation
                break
        else:
            if temp_state == sol_state:
                # print(f'solved id: {id}')
                if len(rotation) > 0:
                    mmoves += '.' + rotation
                break
                
    return temp_state, mmoves

In [7]:
def move_translation(dim):
    M = {}
    M["U"] = f'-d{dim-1}'
    M["R"] = "r0"
    M["B"] = f"-f{dim-1}"
    M["F"] = "f0"
    M["L"] = f"-r{dim-1}"
    M["D"] = "d0"
    
    
    I = ['.'.join([f'{j}{i}' for i in range(dim)]) for j in ['r', 'f', 'd']]
    for i, j in enumerate(['x', 'y', 'z']):
        M[j] = I[i]

    if dim > 3:
        M["Uw"] = f'-d{dim-2}.-d{dim-1}'
        M["Rw"] = f"r0.r1"
        M["Bw"] = f'-f{dim-2}.-f{dim-1}'
        M["Fw"] = f"f0.f1"
        M["Lw"] = f'-r{dim-2}.-r{dim-1}'
        M["Dw"] = f"d0.d1"
        
    if dim >= 6:
        M["2Uw"] = f'-d{dim-2}.-d{dim-1}'
        M["2Rw"] = f"r0.r1"
        M["2Bw"] = f'-f{dim-2}.-f{dim-1}'
        M["2Fw"] = f"f0.f1"
        M["2Lw"] = f'-r{dim-2}.-r{dim-1}'
        M["2Dw"] = f"d0.d1"

        width_max = dim // 2
        for i in range(3, width_max + 1):
            M[f"{i}Uw"] = f'-d{dim-i}.' + M[f"{i-1}Uw"]
            M[f"{i}Rw"] = M[f"{i-1}Rw"] + f'.r{i-1}'
            M[f"{i}Bw"] = f'-f{dim-i}.' + M[f"{i-1}Bw"]
            M[f"{i}Fw"] = M[f"{i-1}Fw"] + f'.f{i-1}'
            M[f"{i}Lw"] = f'-r{dim-i}.' + M[f"{i-1}Lw"]
            M[f"{i}Dw"] = M[f"{i-1}Dw"] + f'.d{i-1}'


    for m in list(M):
        M[m+"2"] = M[m] + '.' + M[m]
        if "-" in M[m]:
            M[m+"'"] = M[m].replace("-","")
        else:
            M[m+"'"] = '.'.join(["-"+i for i in M[m].split('.')])
    
    return M

def reverse_moves(move_seq):
    moves = move_seq.split('.')
    rev_moves = []
    for m in moves[::-1]:
        if m[0] == '-':
            rev_moves.append(m[1:])
        else:
            rev_moves.append(f'-{m}')
    return '.'.join(rev_moves)

In [8]:
def solve(initial_state, solution_state, num_wildcards, moves, verbose=False):
    init_state = relabel_NxNxN_index(initial_state)
    sol_state = relabel_NxNxN_index(solution_state)
    state = state2ubl(init_state)
    # state = ''.join(init_state.split(';'))

    # print(f'Starting {id}')
    output = !./rubiks-cube-solver.py --state $state
    # return output
    print(output[-1])
    sol = None
    if output[-1][:9] == 'Solution:':
        # print(output[-1])
        sol = output[-1].split(': ')[1]
    else:
        for n in range(1, 21):
            if 'Solution:' in output[-n]:
                sol = output[-n].split('Solution: ')[1].split('2023-')[0]
                break
    if sol is None:
        print(output[-1])
        return output
                
    mmoves = '.'.join([M[m] for m in sol.split(' ')])
    
    new_state = apply_moves(init_state, moves, mmoves)
    
    temp_state, mmoves = sym_rotations(dim, mmoves, new_state, sol_state, num_wildcards, wildcard=False)
    
    return mmoves, temp_state

<a id="4"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >SOLVE<br><div> 


### Outline
1. Create base and SGS
2. Use sift to find solution
3. Interpret solution as moves from competition

<a id="4.1"></a>
## <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:#6A1B9A; font-size:100%; text-align:left; padding:3.0px; background: white; border-bottom: 8px solid #9C27B0">JIX REPO<br><div>

In [9]:
# X = [
#    9, 10, 11, 12, 13, 14, 15, 16, 17,
#   45, 46, 47, 48, 49, 50, 51, 52, 53,
#   24, 21, 18, 25, 22, 19, 26, 23, 20,
#    8,  7,  6,  5,  4,  3,  2,  1,  0,
#   38, 41, 44, 37, 40, 43, 36, 39, 42,
#   35, 34, 33, 32, 31, 30, 29, 28, 27,
# ]
# Y = [
#   20, 23, 26, 19, 22, 25, 18, 21, 24,
#   11, 14, 17, 10, 13, 16,  9, 12, 15,
#   47, 50, 53, 46, 49, 52, 45, 48, 51,
#   33, 30, 27, 34, 31, 28, 35, 32, 29,
#    2,  5,  8,  1,  4,  7,  0,  3,  6,
#   38, 41, 44, 37, 40, 43, 36, 39, 42,
# ]
# R = [
#    0,  1, 11,  3,  4, 14,  6,  7, 17,
#    9, 10, 47, 12, 13, 50, 15, 16, 53,
#   24, 21, 18, 25, 22, 19, 26, 23, 20,
#    8, 28, 29,  5, 31, 32,  2, 34, 35,
#   36, 37, 38, 39, 40, 41, 42, 43, 44,
#   45, 46, 33, 48, 49, 30, 51, 52, 27,
# ]
# 
# xyr_gens = [X, Y, R]
# 
# # TODO: needs to be translated for puzzle representation for competition
# center_cubelet_faces = list(range(4, 54, 9))

In [10]:


# grp = Group(Config(monte_carlo=False, schreier_tree='deep'))
# add_rubiks_gens(grp, xyr_gens)
# print_all_stats(grp)
# 
# known_group_order = grp.order()
# 
# print('\n')
# grp = Group(base=center_cubelet_faces)
# print(f'  base perfix = {grp.base()}')
# add_rubiks_gens(grp, xyr_gens)
# grp.build(known_group_order)
# grp.verify()
# print_sgs_stats(grp)

In [11]:
# # interpret generators as allowed moves from competition
# 
# base = grp.base()
# sgs = grp.generators()
# 
# init_state = sgs[0][0]
# sol_state = list(range(len(init_state)))
# 
# 
# id = 205
# row = puzzles.loc[id]
# 
# dim = int(row['puzzle_type'].split('/')[-1])
# moves = get_moves(puzzle_info.loc[row['puzzle_type'], 'allowed_moves'])
# M = move_translation(dim)

# init_moves, solver_moves = solve(init_state, sol_state, 0, moves) 
# 
# init_moves

<a id="4.2"></a>
## <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:#6A1B9A; font-size:100%; text-align:left; padding:3.0px; background: white; border-bottom: 8px solid #9C27B0">My Attempt<br><div>

In [12]:
N = 3
center_cubelet_faces = list(range(((N-1)//2)*(N+1), 6*N**2, N**2))
# allowed moves
move_dict = eval(puzzle_info.loc[f'cube_{N}/{N}/{N}', 'allowed_moves'])
# perm_dict = {k: Permutation(v) for k, v in move_dict.items()}
# cube_group = PermutationGroup(list(perm_dict.values()))
move_dict.keys()

dict_keys(['f0', 'f1', 'f2', 'r0', 'r1', 'r2', 'd0', 'd1', 'd2'])

In [13]:
# apply deep tree to get group order
grp = Group(Config(monte_carlo=False, schreier_tree='deep'))
add_rubiks_gens(grp, list(move_dict.values()))
print_all_stats(grp)
known_group_order = grp.order()

  group order = 1,038,048,078,587,756,544,000
  strong generating set base = [6, 3, 0, 2, 1, 8, 14, 17, 5, 26, 23, 25, 32, 34, 4, 7, 12, 16, 15, 13]
  strong generating set size = 42
  took 297,934 group products


In [14]:

# supply start to base for cube faces
grp = Group(base=center_cubelet_faces)
print(f'  base perfix = {grp.base()}')
add_rubiks_gens(grp, list(move_dict.values()))
grp.build(known_group_order)
grp.verify()
print_sgs_stats(grp)



  base perfix = [4, 13, 22, 31, 40, 49]
  group order = 1,038,048,078,587,756,544,000
  strong generating set base = [4, 13, 22, 31, 40, 49, 6, 0, 2, 15, 1, 3, 5, 7, 8, 12, 14, 16, 17, 23, 25, 26, 32, 34]
  strong generating set size = 33


In [15]:
base = grp.base()
sgs = grp.generators()

id = 205
row = puzzles.loc[id]

perm = [int(i) for i in row['initial_state'].replace("N", "").split(";")]
p_sifted, history = grp.sift_history(perm.copy())
# history[::1]
temp_perm = perm.copy()
print(temp_perm)
for g, inv in history[::-1]:
    temp_perm = [temp_perm[i] for i in sgs[g][inv]]
    
    
print(temp_perm)

[3, 33, 20, 0, 50, 85, 53, 72, 7, 22, 41, 66, 83, 91, 87, 47, 31, 46, 45, 95, 43, 37, 38, 55, 39, 25, 73, 36, 16, 4, 40, 19, 60, 8, 59, 51, 68, 6, 70, 62, 23, 54, 10, 18, 15, 61, 30, 28, 64, 71, 11, 35, 93, 74, 69, 78, 14, 89, 90, 13, 80, 77, 49, 76, 48, 1, 34, 44, 84, 57, 21, 56, 17, 58, 42, 52, 92, 29, 24, 12, 67, 65, 27, 32, 75, 26, 9, 94, 81, 86, 5, 82, 63, 2, 88, 79]
[3, 33, 20, 0, 50, 85, 53, 72, 7, 22, 41, 66, 83, 91, 87, 47, 31, 46, 45, 95, 43, 37, 38, 55, 39, 25, 73, 36, 16, 4, 40, 19, 60, 8, 59, 51, 68, 6, 70, 62, 23, 54, 10, 18, 15, 61, 30, 28, 64, 71, 11, 35, 93, 74, 69, 78, 14, 89, 90, 13, 80, 77, 49, 76, 48, 1, 34, 44, 84, 57, 21, 56, 17, 58, 42, 52, 92, 29, 24, 12, 67, 65, 27, 32, 75, 26, 9, 94, 81, 86, 5, 82, 63, 2, 88, 79]


In [16]:
# sgs_translated = []
# moves = get_moves(move_dict)
# # print(moves)
# for g in sgs:
#     # print(g[0])
#     
#     init_state = g[0]
#     sol_state = list(range(len(init_state)))
#     
#     dim = N
#     
#     M = move_translation(dim)
#     
#     init_moves, transformed_state = solve(init_state, sol_state, 0, moves) 
#     
#     print(init_moves)
#     # print(reverse_moves(init_moves))
#     sgs_translated.append((init_moves, reverse_moves(init_moves)))
#     # print(transformed_state)
#     # break

In [17]:
# sol_moves = []
# for g, inv in history[::-1]:
#     sol_moves.extend(sgs_translated[g][inv].split('.'))
# '.'.join(sol_moves)

In [18]:
move_dict.keys()

dict_keys(['f0', 'f1', 'f2', 'r0', 'r1', 'r2', 'd0', 'd1', 'd2'])

In [19]:
# print(moves)

In [23]:
# supply start to base for cube faces
grp = Group(base=center_cubelet_faces)
print(f'  base perfix = {grp.base()}')
add_rubiks_gens(grp, list(move_dict.values()))
grp.rebuild_schreier_tree()
grp.tree_expand

  base perfix = [4, 13, 22, 31, 40, 49]


[([(0, False)], [(0, True)]),
 ([(1, False)], [(1, True)]),
 ([(2, False)], [(2, True)]),
 ([(3, False)], [(3, True)]),
 ([(4, False)], [(4, True)]),
 ([(5, False)], [(5, True)]),
 ([(6, False)], [(6, True)]),
 ([(7, False)], [(7, True)]),
 ([(8, False)], [(8, True)])]

In [21]:
# grp.build(known_group_order)
# grp.verify()
# print_sgs_stats(grp)